In [249]:
import pandas as pd
from tqdm import tqdm
import requests
from sqlite3 import connect
import time
from concurrent.futures import ThreadPoolExecutor

In [250]:
# Load data
season = 2025
season_string = 20242025
shots = pd.read_csv(f'./shots/{season}_shots_g.csv', encoding='latin-1')
goals = pd.read_csv(f'{season_string}_goals.csv')

In [251]:
print(len(shots))
shots = shots[shots["goalie"] != "ENG"]
print(len(shots))

9224
9224


In [252]:
shots['team_skaters'] = shots['team_skaters'].astype(str)
shots['opposing_skaters'] = shots['opposing_skaters'].astype(str)

In [253]:
def split_skaters(skaters):
    skaters = skaters.split(',')
    return [x.strip(' ') for x in skaters]

shots['opposing_skaters'] = shots.apply(lambda row: split_skaters(row['opposing_skaters']), axis=1)
shots['team_skaters'] = shots.apply(lambda row: split_skaters(row['team_skaters']), axis=1)

shots['team_skaters']

0       [8474593, 8480002, 8476292, 8483495, 8475193, ...
1       [8474593, 8478414, 8479407, 8476462, 8481559, ...
2       [8480045, 8482671, 8481524, 8479420, 8477949, ...
3           [8474593, 8480002, 8476462, 8482110, 8478399]
4       [8480802, 8480807, 8475722, 8480045, 8482671, ...
                              ...                        
9219    [8474593, 8478414, 8476462, 8475287, 8482110, ...
9220    [8474593, 8480002, 8478414, 8477488, 8482684, ...
9221    [8481668, 8480073, 8480748, 8478454, 8478046, ...
9222    [8481668, 8480073, 8480748, 8478454, 8478046, ...
9223    [8474593, 8481032, 8478414, 8477488, 8475287, ...
Name: team_skaters, Length: 9224, dtype: object

In [254]:
# Create dictionaries for skater statistics
skaters = {}

for i in tqdm(range(0, len(shots))):
    shot = shots.iloc[i]
    if shot['strength'] != 'EV':
        continue
    # xGA
    for skater_id in shot['opposing_skaters']:
        skater_dict = skaters.get(skater_id, {'xGF': 0, 'xGA': 0, 'primary_assists_PP': 0, 'primary_assists_EV': 0, 'sh_xGA': 0})
        skater_dict['xGA'] += shot['xGF']
        skaters[skater_id] = skater_dict

    # xGF
    for skater_id in shot['team_skaters']:
        skater_dict = skaters.get(skater_id, {'xGF': 0, 'xGA': 0, 'primary_assists_PP': 0, 'primary_assists_EV': 0, 'sh_xGA': 0})
        skater_dict['xGF'] += shot['xGF']
        skaters[skater_id] = skater_dict

skaters

100%|██████████| 9224/9224 [00:00<00:00, 16371.22it/s]


{'8480035': {'xGF': 5.323924450000001,
  'xGA': 5.683953079999999,
  'primary_assists_PP': 0,
  'primary_assists_EV': 0,
  'sh_xGA': 0},
 '8480839': {'xGF': 7.1780815480000015,
  'xGA': 5.841777625999998,
  'primary_assists_PP': 0,
  'primary_assists_EV': 0,
  'sh_xGA': 0},
 '8480045': {'xGF': 11.73493202599999,
  'xGA': 13.506596362999995,
  'primary_assists_PP': 0,
  'primary_assists_EV': 0,
  'sh_xGA': 0},
 '8477979': {'xGF': 0.8256486759999999,
  'xGA': 0.251134468,
  'primary_assists_PP': 0,
  'primary_assists_EV': 0,
  'sh_xGA': 0},
 '8478043': {'xGF': 2.012495716,
  'xGA': 2.308352702,
  'primary_assists_PP': 0,
  'primary_assists_EV': 0,
  'sh_xGA': 0},
 '8479359': {'xGF': 2.8837518869999994,
  'xGA': 2.3856235910000003,
  'primary_assists_PP': 0,
  'primary_assists_EV': 0,
  'sh_xGA': 0},
 '8474593': {'xGF': 19.610404502925245,
  'xGA': 15.201348635811192,
  'primary_assists_PP': 0,
  'primary_assists_EV': 0,
  'sh_xGA': 0},
 '8480002': {'xGF': 9.496524285047848,
  'xGA': 5.52

In [255]:
skaters.pop('nan', None)

In [256]:
limit = 100
start = 0
all_players = []

while True:
    url = f"https://api.nhle.com/stats/rest/en/skater/timeonice?isAggregate=false&isGame=false&sort=%5B%7B%22property%22:%22timeOnIce%22,%22direction%22:%22DESC%22%7D,%7B%22property%22:%22playerId%22,%22direction%22:%22ASC%22%7D%5D&start={start}&limit={limit}&factCayenneExp=gamesPlayed%3E=1&cayenneExp=gameTypeId=2%20and%20seasonId%3C={season_string}%20and%20seasonId%3E={season_string}"
    response = requests.get(url)
    data = response.json()

    # Check if there are more records to retrieve
    if not data["data"]:
        break

    # Add the current batch of players to the list
    all_players.extend(data["data"])

    # Increment the start parameter for the next request
    start += limit

In [257]:
def update_stats(player_id):
    stats = skaters[player_id]
    
    while True:
        try:
            player = requests.get(f'https://api-web.nhle.com/v1/player/{player_id}/landing').json()
            break
        except (requests.exceptions.RequestException, ConnectionResetError) as err:
            time.sleep(10)
            print('Taking a break...')

    # Populate Dictionary with relevant data
    stats['firstName'] = player['firstName']["default"]
    stats['lastName'] = player['lastName']["default"]
    stats['birthDate'] = player['birthDate']
    stats['nationality'] = player['birthCountry']
    stats['position'] = player['position']

    if stats['position'] == 'G':
        return
    stats['height'] = player['heightInInches']
    stats['weight'] = player['weightInPounds']  
    stats['team'] = list(filter(lambda x: x['leagueAbbrev'] == 'NHL' and x["season"] == season_string, player['seasonTotals']))[-1]['teamName']["default"]
    stats['handedness'] = player['shootsCatches']

    for player_ in all_players:
        if int(player_["playerId"]) == int(player_id):
            filtered_player = player_
            break
        else:
            filtered_player = None
    if filtered_player:
        toi_stats = filtered_player
    else:
        toi_stats = None
        


    # TOI
    if toi_stats is not None: 
        stats['TOI'] = toi_stats['timeOnIcePerGame'] /60 
        stats['EV_TOI'] = toi_stats['evTimeOnIcePerGame'] /60
        stats['PP_TOI'] = toi_stats['ppTimeOnIcePerGame'] /60
        stats['SH_TOI'] = toi_stats['shTimeOnIcePerGame'] /60
        stats['TOTAL_TOI'] = toi_stats['timeOnIce'] /60
        stats['TOTAL_EV_TOI'] = toi_stats['evTimeOnIce'] /60
        stats['TOTAL_PP_TOI'] = toi_stats['ppTimeOnIce'] /60
        stats['TOTAL_SH_TOI'] = toi_stats['shTimeOnIce'] /60
        stats['games'] = toi_stats['gamesPlayed']


        stats['xGF'] = stats['xGF'] / stats['TOTAL_EV_TOI'] * 60 if stats['TOTAL_EV_TOI'] != 0 else 0
        stats['xGA'] = stats['xGA'] / stats['TOTAL_EV_TOI'] * 60 if stats['TOTAL_EV_TOI'] != 0 else 0

        # Rate statistics
        nhl_seasons = list(filter(lambda x: x['leagueAbbrev'] == 'NHL' and x["season"] == season_string, player['seasonTotals']))
        goals = 0
        sh_goals = 0
        pp_goals = 0
        pp_points = 0
        for seas in nhl_seasons:
            goals += seas["goals"]
            sh_goals += seas["shorthandedGoals"]
            pp_goals += seas["powerPlayGoals"]
            pp_points += seas["powerPlayPoints"]
        
        stats['goals_60'] = (goals - sh_goals - pp_goals) / stats['TOTAL_EV_TOI'] * 60 if stats['TOTAL_EV_TOI'] != 0 else 0
        stats['pp_60'] = pp_points / stats['TOTAL_PP_TOI'] * 60 if stats['TOTAL_PP_TOI'] != 0 else 0

        skaters[player_id] = stats

with ThreadPoolExecutor(max_workers=20) as executor:
    list(tqdm(executor.map(update_stats, skaters.keys()), total=len(skaters)))

100%|██████████| 754/754 [00:13<00:00, 56.84it/s]


In [258]:
# Shorthanded Rate

for i in tqdm(range(0, len(shots))):
    shot = shots.iloc[i]
    if shot['strength'] != 'Shorthanded':
        continue
    # xGA
    for skater_id in shot['opposing_skaters']:
        skater_dict = skaters.get(skater_id)
        if skater_dict == None:
            continue
        skater_dict['sh_xGA'] += shot['xGF']
        skaters[skater_id] = skater_dict

for player_id, stats in skaters.items():
    if stats['position'] == 'G':
        continue
    if 'TOTAL_SH_TOI' not in stats.keys():
        stats['sh_xGA'] = 0
        continue
    stats['sh_xGA'] = stats['sh_xGA'] / stats['TOTAL_SH_TOI']if stats['TOTAL_SH_TOI'] != 0 else 0
    stats['sh_xGA'] *= 60
    skaters[player_id] = stats

skaters

100%|██████████| 9224/9224 [00:00<00:00, 26310.87it/s]


{'8480035': {'xGF': 1.9179553707595316,
  'xGA': 2.047656468327829,
  'primary_assists_PP': 0,
  'primary_assists_EV': 0,
  'sh_xGA': 0.0,
  'firstName': 'Henri',
  'lastName': 'Jokiharju',
  'birthDate': '1999-06-17',
  'nationality': 'FIN',
  'position': 'D',
  'height': 72,
  'weight': 200,
  'team': 'Buffalo Sabres',
  'handedness': 'R',
  'TOI': 18.465,
  'EV_TOI': 16.654999999999998,
  'PP_TOI': 0.25166666666666665,
  'SH_TOI': 1.5583333333333333,
  'TOTAL_TOI': 184.65,
  'TOTAL_EV_TOI': 166.55,
  'TOTAL_PP_TOI': 2.5166666666666666,
  'TOTAL_SH_TOI': 15.583333333333334,
  'games': 10,
  'goals_60': 0.36025217652356645,
  'pp_60': 0.0},
 '8480839': {'xGF': 2.1930827100738357,
  'xGA': 1.7848085762199775,
  'primary_assists_PP': 0,
  'primary_assists_EV': 0,
  'sh_xGA': 5.268174382869379,
  'firstName': 'Rasmus',
  'lastName': 'Dahlin',
  'birthDate': '2000-04-13',
  'nationality': 'SWE',
  'position': 'D',
  'height': 75,
  'weight': 204,
  'team': 'Buffalo Sabres',
  'handedness'

In [259]:
# Primary Assist Contribution
for i in tqdm(range(0, len(goals))):
    if str(goals.iloc[i]['Assist 1']) != 'nan':
        player_id = str(goals.iloc[i]['Assist 1'])[:7]
        if player_id == '8484720' or player_id =='8482018':
            continue
        stats = skaters[player_id]
        if stats['position'] == 'G':
            continue
        if goals.iloc[i]['strength'] == 'EV':
            stats['primary_assists_EV'] += 1
        elif goals.iloc[i]['strength'] == 'Powerplay':
            stats['primary_assists_PP'] += 1
        
        skaters[player_id] = stats
    
for player_id, stats in tqdm(skaters.items()):
    if stats['position'] == 'G':
        continue
    if 'TOTAL_EV_TOI' not in stats.keys():
        stats['primary_assists_EV'] = 0
        stats['primary_assists_PP'] = 0
        continue
    stats['primary_assists_EV'] = stats['primary_assists_EV'] / (stats['TOTAL_EV_TOI']) if stats['TOTAL_EV_TOI'] != 0 else 0
    stats['primary_assists_EV'] *= 60

    stats['primary_assists_PP'] = stats['primary_assists_PP'] /(stats['TOTAL_PP_TOI']) if stats['TOTAL_PP_TOI'] != 0 else 0
    stats['primary_assists_PP'] *= 60

    skaters[player_id] = stats

100%|██████████| 754/754 [00:00<00:00, 753874.90it/s]


In [260]:
# Get WPA Per Player
goals = shots[shots['isGoal'] == 1]
for i in tqdm(range(0, len(goals))):
    shot = goals.iloc[i]
    shooter_id = str(int(shot['shooter_id']))[:7]

    stats = skaters[shooter_id]
    stats['wpa'] = stats.get('wpa', 0)
    stats['wpa'] += shot['win_prob_added']
    skaters[shooter_id] = stats
    if not pd.isna(shot['primary_assist']):
        assist_id = str(int(shot['primary_assist']))[:7]
        assist_stats = skaters[assist_id]
        assist_stats['wpa'] = assist_stats.get('wpa', 0)
        assist_stats['wpa'] += shot['win_prob_added']
        skaters[assist_id] = assist_stats

100%|██████████| 946/946 [00:00<00:00, 22425.50it/s]


In [261]:
# # Get Shots Per Player
# account_id = 6415718365001
# shots_goals = shots[shots['isGoal'] == 1]
# for i in tqdm(range(0, len(shots_goals))):
#     shot = shots_goals.iloc[i]

#     stats = skaters[str(int(shot['shooter_id']))[:7]]

#     if stats['position'] == 'G':
#         continue

#     player_shots = stats.get('shots', [])

#     event_id = str(shot['shot_id'])
#     game = shot['gameID']
#     link = ['No link found.']

#     # Grab media (shoutout trusty)
#     if event_id != 'nan':
#         failed = True
#         while failed:
#             try:
#                 media = requests.get(f'https://forge-dapi.d3.nhle.com/v2/content/en-US/videos?context.slug=nhl&tags.slug=highlight&tags.slug=gameid-{game}').json()
#                 failed = False
#             except (requests.exceptions.RequestException,
#                 ConnectionResetError) as err:
#                 time.sleep(20)
#                 print('Taking a break...')
#         highlights =  media["items"]
#         shot_list = list(filter(lambda x: "fields" in x.keys(), highlights))
#         shot_list = list(map(lambda x: x["fields"], shot_list))
#         shot_list = list(filter(lambda x: "statsEventId" in x.keys() and x["statsEventId"] == event_id, shot_list))
#         if len(shot_list) != 0:
#             shot_id = shot_list[0]['statsEventId']
#             if str(event_id) == shot_id:
#                 try: 
#                     link = f'https://players.brightcove.net/6415718365001/EXtG1xJ7H_default/index.html?videoId={shot_list[0]["brightcoveId"]}'
#                 except:
#                     # Do nothing
#                     link

#     shooter = shot['shooter']
#     if shooter == "K'Andre Miller":
#         print('Unserious')
#         shooter = "KAndre Miller"

#     player_shots.append({'date': shot['date'], 'shooter': shooter, 'goalie': shot['goalie'], 'strength': shot['strength'], 'away_goals': shot['away_goals'], 'home_goals': shot['home_goals'], 'home_team': shot['home'], 'away_team': shot['away'], 'x': shot['x'], 'y': shot['y'], 'prev_event': shot['prev_event'], 'type': shot['shotType'], 'xG': shot['xGF'], 'Outcome': shot['event'], 'Link': link})

#     stats['shots'] = player_shots
#     skaters[str(int(shot['shooter_id']))[:7]] = stats

In [262]:
# # Get Assists Per Player
# shots_goals = shots[shots['isGoal'] == 1]
# for i in tqdm(range(0, len(shots_goals))):
#     shot = shots_goals.iloc[i]
#     if str(shot['primary_assist']) == 'nan':
#         continue
#     stats = skaters[str(int(shot['primary_assist']))[:7]]

#     if stats['position'] == 'G':
#         continue

#     player_shots = stats.get('assists', [])

#     event_id = str(shot['shot_id'])
#     game = shot['gameID']
#     link = ['No link found.']

#     # Grab media (shoutout trusty)
#     if event_id != 'nan':
#         failed = True
#         while failed:
#             try:
#                 media = requests.get(f'https://forge-dapi.d3.nhle.com/v2/content/en-US/videos?context.slug=nhl&tags.slug=highlight&tags.slug=gameid-{game}').json()
#                 failed = False
#             except (requests.exceptions.RequestException,
#                 ConnectionResetError) as err:
#                 time.sleep(20)
#                 print('Taking a break...')
#         highlights =  media["items"]
#         shot_list = list(filter(lambda x: "fields" in x.keys(), highlights))
#         shot_list = list(map(lambda x: x["fields"], shot_list))
#         shot_list = list(filter(lambda x: "statsEventId" in x.keys() and x["statsEventId"] == event_id, shot_list))
#         if len(shot_list) != 0:
#             shot_id = shot_list[0]['statsEventId']
#             if str(event_id) == shot_id:
#                 try:
#                     link = f'https://players.brightcove.net/6415718365001/EXtG1xJ7H_default/index.html?videoId={shot_list[0]["brightcoveId"]}'
#                 except:
#                     link

#     shooter = shot['shooter']
#     if shooter == "K'Andre Miller":
#         print('Unserious')
#         shooter = "KAndre Miller"

    


#     player_shots.append({'date': shot['date'], 'shooter': shooter, 'goalie': shot['goalie'], 'strength': shot['strength'], 'away_goals': shot['away_goals'], 'home_goals': shot['home_goals'], 'home_team': shot['home'], 'away_team': shot['away'], 'x': shot['x'], 'y': shot['y'], 'prev_event': shot['prev_event'], 'type': shot['shotType'], 'xG': shot['xGF'], 'Outcome': shot['event'], 'Link': link})

#     stats['assists'] = player_shots
#     skaters[str(int(shot['primary_assist']))[:7]] = stats

In [263]:

for player_id, stats in tqdm(skaters.items()):
    player_shots = shots[shots['shooter_id'].astype(str).str[:7] == str(player_id)]
    
    xG = player_shots['xGF'].sum()
    goals = player_shots['isGoal'].sum()
    
    stats['gsae'] = goals - xG

100%|██████████| 754/754 [00:03<00:00, 211.80it/s]


In [264]:
# Convert Dict to Pandas

players = []

for player_id, stats in skaters.items():
    if player_id == "nan":
        continue
    str_id = str(int(player_id))[:7]
    if len(str_id) != 7:
        continue
    stats['player_id'] = str_id
    players.append(pd.DataFrame.from_dict([stats]))

df = pd.concat(players)
df['season'] = [season] * len(df)
    

In [265]:
conn = connect("player_stats.db")
curr = conn.cursor()

# create a table 
curr.execute('CREATE TABLE IF NOT EXISTS skaters (xGF NUMBER, xGA NUMBER, primary_assists_pp NUMBER, primary_assists_EV NUMBER, sh_xGA NUMBER, firstName TEXT, lastName TEXT, birthDate TEXT, nationality TEXT, position TEXT, height TEXT, weight NUMBER, team TEXT, handedness TEXT, TOI NUMBER, EV_TOI NUMBER, PP_TOI NUMBER, SH_TOI NUMBER, goals_60 NUMBER, pp_60 NUMBER, gsae NUMBER, wpa NUMBER, player_id TEXT, season NUMBER)')
 
# commit the query
conn.commit()

In [266]:
df.dtypes

xGF                   float64
xGA                   float64
primary_assists_PP    float64
primary_assists_EV    float64
sh_xGA                float64
firstName              object
lastName               object
birthDate              object
nationality            object
position               object
height                float64
weight                float64
team                   object
handedness             object
TOI                   float64
EV_TOI                float64
PP_TOI                float64
SH_TOI                float64
TOTAL_TOI             float64
TOTAL_EV_TOI          float64
TOTAL_PP_TOI          float64
TOTAL_SH_TOI          float64
games                 float64
goals_60              float64
pp_60                 float64
wpa                   float64
gsae                  float64
player_id              object
season                  int64
dtype: object

In [267]:
# Make sure types match up
df['firstName'] = df['firstName'].astype(str)
df['lastName'] = df['lastName'].astype(str)
df['birthDate'] = df['birthDate'].astype(str)
df['nationality'] = df['nationality'].astype(str)
df['height'] = df['height'].astype(str)
df['team'] = df['team'].astype(str)
df['handedness'] = df['handedness'].astype(str)
df['player_id'] = df['player_id'].astype(str)

In [268]:
# Ice Time Minimums
df = df[df['TOTAL_TOI'] >= 50]
df.drop(['TOTAL_TOI', 'games', 'TOTAL_EV_TOI', 'TOTAL_PP_TOI', 'TOTAL_SH_TOI'], inplace=True, axis=1)

In [269]:
conn.close()
conn = connect("player_stats.db")
curr = conn.cursor()
# create a table 
curr.execute('CREATE TABLE IF NOT EXISTS skaters_no_percentile (season NUMBER, xGF NUMBER, xGA NUMBER, primary_assists_pp NUMBER, primary_assists_EV NUMBER, sh_xGA NUMBER, firstName TEXT, lastName TEXT, birthDate TEXT, nationality TEXT, position TEXT, height TEXT, weight NUMBER, team TEXT, handedness TEXT, TOI NUMBER, EV_TOI NUMBER, PP_TOI NUMBER, SH_TOI NUMBER, goals_60 NUMBER, pp_60 NUMBER, gsae NUMBER, wpa NUMBER, player_id TEXT)')
 
dff = df.copy()
# commit the query
conn.commit()
dff.drop_duplicates(subset=['player_id'], inplace=True)
curr.execute("DELETE FROM skaters_no_percentile WHERE season = 2025")
dff.to_sql('skaters_no_percentile', conn, if_exists='append', index=False)

611

In [270]:
conn.close()

In [271]:
# Split players by forward and defensemen
forwards = df[df['position'] != 'D']
df = df[df['position'] == 'D']

In [272]:
# Convert statistics to percentile
forwards['xGF'] = forwards.xGF.rank(pct = True)
forwards['xGA'] = forwards.xGA.rank(pct = True, ascending = False)
forwards['primary_assists_PP'] = forwards.primary_assists_PP.rank(method= 'min', pct = True)
forwards['primary_assists_EV'] = forwards.primary_assists_EV.rank(method= 'min', pct = True)
forwards['sh_xGA'] = forwards.sort_values(['SH_TOI'], ascending=False)['sh_xGA'].rank(method= 'first', pct = True, ascending = False)
forwards['TOI'] = forwards.TOI.rank(pct = True)
forwards['EV_TOI'] = forwards.EV_TOI.rank(pct = True)
forwards['PP_TOI'] = forwards.PP_TOI.rank(method='min', pct = True)
forwards['SH_TOI'] = forwards.SH_TOI.rank(method='min', pct = True)
forwards['goals_60'] = forwards.goals_60.rank(method='min', pct = True)
forwards['pp_60'] = forwards.pp_60.rank(method= 'min', pct = True)
forwards['gsae'] = forwards.gsae.rank(pct = True)

df['xGF'] = df.xGF.rank(pct = True)
df['xGA'] = df.xGA.rank(pct = True, ascending = False)
df['primary_assists_PP'] = df.primary_assists_PP.rank(method= 'min', pct = True)
df['primary_assists_EV'] = df.primary_assists_EV.rank(method= 'min', pct = True)
df['sh_xGA'] = df.sort_values(['SH_TOI'], ascending=False)['sh_xGA'].rank(method= 'first', pct = True, ascending = False)
df['TOI'] = df.TOI.rank(pct = True)
df['EV_TOI'] = df.EV_TOI.rank(pct = True)
df['PP_TOI'] = df.PP_TOI.rank(method='min', pct = True)
df['SH_TOI'] = df.SH_TOI.rank(method='min', pct = True)
df['goals_60'] = df.goals_60.rank(method='min', pct = True)
df['pp_60'] = df.pp_60.rank(method= 'min', pct = True)
df['gsae'] = df.gsae.rank(pct = True)

In [273]:
# Combine

df = pd.concat([df, forwards], ignore_index=False)

In [274]:
# Dump to SQL
conn = connect("player_stats.db")
curr = conn.cursor()
df.drop_duplicates(subset=['player_id'], inplace=True)
curr.execute("DELETE FROM skaters WHERE season = 2025")
df.to_sql('skaters', conn, if_exists='append', index=False)

611

In [275]:

# curr.execute("ALTER TABLE goalies RENAME TO goalies_2023")
# curr.execute("ALTER TABLE players RENAME TO players_2023")
# curr.execute("ALTER TABLE players_no_percentile RENAME TO players_no_percentile_2023")

In [276]:
# # curr.execute("CREATE TABLE teams (team_id INTEGER PRIMARY KEY AUTOINCREMENT,team_name TEXT NOT NULL,location TEXT,name TEXT,team_abbr TEXT,division TEXT,conference TEXT,primary_color TEXT,secondary_color TEXT);")
# curr.execute("INSERT INTO teams (team_name, location, name, team_abbr, division, conference, primary_color, secondary_color) VALUES ('Anaheim Ducks', 'Anaheim', 'Ducks', 'ANA', 'Pacific', 'Western', '#B9975B', '#000000'), ('Arizona Coyotes', 'Arizona', 'Coyotes', 'ARI', 'Central', 'Western', '#8C2633', '#E2D6B5'), ('Boston Bruins', 'Boston', 'Bruins', 'BOS', 'Atlantic', 'Eastern', '#FFB81C', '#000000'), ('Buffalo Sabres', 'Buffalo', 'Sabres', 'BUF', 'Atlantic', 'Eastern', '#002654', '#FDBB30'), ('Calgary Flames', 'Calgary', 'Flames', 'CGY', 'Pacific', 'Western', '#C8102E', '#F1BE48'), ('Carolina Hurricanes', 'Raleigh', 'Hurricanes', 'CAR', 'Metropolitan', 'Eastern', '#CC0000', '#000000'), ('Chicago Blackhawks', 'Chicago', 'Blackhawks', 'CHI', 'Central', 'Western', '#CF0A2C', '#000000'), ('Colorado Avalanche', 'Denver', 'Avalanche', 'COL', 'Central', 'Western', '#6F263D', '#236192'), ('Columbus Blue Jackets', 'Columbus', 'Blue Jackets', 'CBJ', 'Metropolitan', 'Eastern', '#002654', '#CE1126'), ('Dallas Stars', 'Dallas', 'Stars', 'DAL', 'Central', 'Western', '#00843D', '#000000'), ('Detroit Red Wings', 'Detroit', 'Red Wings', 'DET', 'Atlantic', 'Eastern', '#CE1126', '#FFFFFF'), ('Edmonton Oilers', 'Edmonton', 'Oilers', 'EDM', 'Pacific', 'Western', '#FF4C00', '#003876'), ('Florida Panthers', 'Sunrise', 'Panthers', 'FLA', 'Atlantic', 'Eastern', '#041E42', '#C8102E'), ('Los Angeles Kings', 'Los Angeles', 'Kings', 'LAK', 'Pacific', 'Western', '#A2AAAD', '#000000'), ('Minnesota Wild', 'Saint Paul', 'Wild', 'MIN', 'Central', 'Western', '#154734', '#DDCBA4'), ('Montreal Canadiens', 'Montreal', 'Canadiens', 'MTL', 'Atlantic', 'Eastern', '#AF1E2D', '#192168'), ('Nashville Predators', 'Nashville', 'Predators', 'NSH', 'Central', 'Western', '#FFB81C', '#041E42'), ('New Jersey Devils', 'Newark', 'Devils', 'NJD', 'Metropolitan', 'Eastern', '#CE1126', '#000000'), ('New York Islanders', 'Uniondale', 'Islanders', 'NYI', 'Metropolitan', 'Eastern', '#00539B', '#F47D30'), ('New York Rangers', 'New York', 'Rangers', 'NYR', 'Metropolitan', 'Eastern', '#0033A0', '#CE1126'), ('Ottawa Senators', 'Ottawa', 'Senators', 'OTT', 'Atlantic', 'Eastern', '#E31837', '#C69214'), ('Philadelphia Flyers', 'Philadelphia', 'Flyers', 'PHI', 'Metropolitan', 'Eastern', '#F74902', '#000000'), ('Pittsburgh Penguins', 'Pittsburgh', 'Penguins', 'PIT', 'Metropolitan', 'Eastern', '#FCB514', '#000000'), ('San Jose Sharks', 'San Jose', 'Sharks', 'SJS', 'Pacific', 'Western', '#006D75', '#EA7200'), ('Seattle Kraken', 'Seattle', 'Kraken', 'SEA', 'Pacific', 'Western', '#001628', '#99D9D9'), ('St. Louis Blues', 'St. Louis', 'Blues', 'STL', 'Central', 'Western', '#002F87', '#FFD100'), ('Tampa Bay Lightning', 'Tampa', 'Lightning', 'TBL', 'Atlantic', 'Eastern', '#00205B', '#FFFFFF'), ('Toronto Maple Leafs', 'Toronto', 'Maple Leafs', 'TOR', 'Atlantic', 'Eastern', '#00205B', '#FFFFFF'), ('Vancouver Canucks', 'Vancouver', 'Canucks', 'VAN', 'Pacific', 'Western', '#00205B', '#00843D'), ('Vegas Golden Knights', 'Paradise', 'Golden Knights', 'VGK', 'Pacific', 'Western', '#B4975A', '#333F42'),('Washington Capitals', 'Washington', 'Capitals', 'WSH', 'Metropolitan', 'Eastern', '#041E42', '#C8102E'),('Winnipeg Jets', 'Winnipeg', 'Jets', 'WPG', 'Central', 'Western', '#041E42', '#004C97');")



In [277]:
conn.commit()

In [278]:
conn.close()

In [279]:
# conn = connect("player_stats.db")
# curr = conn.cursor()
# curr.execute('''
# DELETE FROM teams WHERE team_id > 32
# ''')
